## Project Summary


This project involves building a machine learning model to predict FIFA player ratings based on hsitorical player data. The process includes data preparation, feature selection, model training, evaluation and deployment. By levaraging cross-validatio abd optimization techniques, the model aims to accurately predict player ratings even on unseen data from a different season. The final step involves deploying the model on a web platform to make it accessible and user-friendly, demonstrating its functionality via video tutorial.

This structured approach ensures that each phase of the project from data handling to deployment is handled systematically to achieve accurate predictions and a functional end product.

In [1]:
import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
players_22 = pd.read_csv('players_22.csv')
male_player =pd.read_csv('male_players (legacy).csv')

C:\Users\hkrma\AppData\Local\Temp\ipykernel_22448\161522061.py:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players_22 = pd.read_csv('players_22.csv')
C:\Users\hkrma\AppData\Local\Temp\ipykernel_22448\161522061.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  male_player =pd.read_csv('male_players (legacy).csv')


## Import Explanation
1. pandas('import pandas as pd'):
   Pandas is a powerful data analysis and manipulation library for Python. it provides data structures like DataFrame and Series,       which are essential for working with structured data.
   
2. numpy('import numpy as np'):
   Numpy is a fundamental package for numerical computing in Python. It provides support for large, multi-dimensional functions to      operate on these arrays.

3. joblib ('import joblub'):
   Joblib is alibrary for saving and loading Python objects(serialized objects). It is particularly useful for saving scikit-learn      modelx after training for later use.

4. train_test_split ('from sklearn.model_selection import train_test_split'):
   'train_test_split' is a function from scikit-learn that splits arrays or matrices into random train and test subsets. This is        useful for splitting your dataset into training and testing sets for model validation.

5. StandardScaler (from sklearn.preprocessing import StandardScaler):
   This is a class form the scikit-learn used for standardizing features by removing the mean and scaling to unit variance. It ensures that each feature in the input data has a mean of 0 and a standard deviation of 1


## Exploratory Data Analysis
This is crucial because it is used to understand the structure, relationship, and distribution of data.

In [3]:
# EDA
print(male_player.head()) #prints out the first 5 rows
print(male_player.info()) #provides a concise summary of the dataframe.
print(male_player.describe()) #provides a summary statistic of the data.

   player_id                                        player_url  fifa_version  \
0     158023                /player/158023/lionel-messi/150002            15   
1      20801  /player/20801/c-ronaldo-dos-santos-aveiro/150002            15   
2       9014                  /player/9014/arjen-robben/150002            15   
3      41236           /player/41236/zlatan-ibrahimovic/150002            15   
4     167495                /player/167495/manuel-neuer/150002            15   

   fifa_update fifa_update_date         short_name  \
0            2       2014-09-18           L. Messi   
1            2       2014-09-18  Cristiano Ronaldo   
2            2       2014-09-18          A. Robben   
3            2       2014-09-18     Z. Ibrahimović   
4            2       2014-09-18           M. Neuer   

                             long_name player_positions  overall  potential  \
0       Lionel Andrés Messi Cuccittini               CF       93         95   
1  Cristiano Ronaldo dos Santos Avei

# 1.Demonstrate the data preparation & feature extraction process [5]


## Function Breakdownm
@param: data - represents the entire dataset.
        d_num - represents the subset of the dataset that contaisn only numeric feartures.
        
@body [Standard Scaling for Numerical Data]:

        Standard Scaler() - standardizes features by removing the mean and scaling to unit variance.

        
        scaler.fit_transform(d_num) - This fits the scaler to the numerical data (d_num) and transforms it. 

        
        dataset_num_scaled_df - converts the scaled numpy array back into a pandas Dataframe.


        
        

In [4]:
def preprocessing(d_num):
    # Standard scaling for numerical data
    scaler = StandardScaler()
    d_num_scaled = scaler.fit_transform(d_num)
    
    # Convert the scaled numerical data back to a DataFrame
    dataset_num_scaled_df = pd.DataFrame(d_num_scaled, columns=d_num.columns)
    

    return dataset_num_scaled_df
    
'''The preprocessing function performs standard scaling on numerical data '''

'The preprocessing function performs standard scaling on numerical data and saves the scaler object for future use(i.e in the deployment stage)'

## Data Cleaning
The data cleaning function performs initial data cleaning tasks on the given dataset. It drops all columsn that have more than 20 % missing valeus, separates numerical and categorial columns and fills missing vlaues in numericla columns with their means. The preprocessing function is then called.

In [5]:
def data_cleaning(dataset):
    # Drop columns with more than 20% missing values
    columns_to_drop = []
    for col in dataset.columns:
        missing = dataset[col].isnull().sum()
        if (missing / len(dataset[col])) * 100 > 20:
            columns_to_drop.append(col)
    dataset.drop(columns=columns_to_drop, inplace=True)
    
    # Separate numerical and categorical columns
    dataset_num = dataset.select_dtypes(include=[np.number])
    dataset_cat = dataset.select_dtypes(include=['object'])

    '''Looking at the function in question, the categorial data is separated from the datat set and dropped. This is because most of the categorial data in the 
    dataset have little to no correlation to the Y values "overall". Also, encoding these values will lead to the creation of multiple columns leading to longer runtimes and lower e
    efficiency of the model. Therefore for this model, all categorial columns where dropped, leaving the prediction of the overall to be based on numerical data.'''
    
    # Fill missing values in numerical data with mean
    dataset_num.fillna(dataset_num.mean(), inplace=True)
    
    return dataset_num

In [6]:
update_22 = data_cleaning(players_22)# cleaned test dataset
male_player_clean = data_cleaning(male_player) # cleaned train dataset

In [7]:
male_player_clean # display of cleaned dataset

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,mentality_vision,mentality_penalties,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,15,2,93,95,100500000.0,550000.0,27,169,67,...,90,76,25,21,20,6,11,15,14,8
1,20801,15,2,92,92,79000000.0,375000.0,29,185,80,...,81,85,22,31,23,7,11,15,14,11
2,9014,15,2,90,90,54500000.0,275000.0,30,180,80,...,84,80,29,26,26,10,8,11,5,15
3,41236,15,2,90,90,52500000.0,275000.0,32,195,95,...,83,91,25,41,27,13,15,10,9,12
4,167495,15,2,90,90,63500000.0,300000.0,28,193,92,...,20,37,25,25,25,87,85,92,90,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,23,2,46,61,110000.0,700.0,18,180,73,...,42,54,23,21,25,9,13,13,12,7
161579,269019,23,2,46,58,110000.0,750.0,19,188,83,...,31,31,50,51,45,6,14,8,13,14
161580,271093,23,2,46,58,110000.0,500.0,19,181,73,...,40,37,36,45,50,8,9,7,14,9
161581,271555,23,2,46,70,150000.0,500.0,17,175,68,...,44,63,19,17,14,13,12,14,7,13


## 2. Create feature subsets that show maximum correlation with the dependent variable. [5]

## Correlation and Feature Engineering 
The function helps identify the most influential features related to the target variable, which can be useful for feature selction in the machine learning .

# Function Breakdown:
Compute Correlation Matrix: The correlation matrix shows how each pair of variables is related.


Correlation with Target Variable: By focusing on the overall column, you can see which features are most strongly related to the target variable.


Select High Correlation Variables: Variables with a correlation greater than 0.5 are considered to have a strong relationship with the target variable.


Remove Target Variable: The target variable overall is excluded from the list since it is not a predictor variable.

In [8]:
def correlation(dataset):
    corr_matrix = dataset.corr() #calculates correlation of all columns in the dataset
    corr_with_overall = corr_matrix['overall'].abs().sort_values(ascending=False)  #extracts the correlation values of all features with teh target variable 'overall'. The absolute values of the correlations are taken to consider both positive and negative correlations.
    preciding_variables = corr_with_overall[corr_with_overall > 0.5].index.tolist()  # selects the features that have an abs correlation greater than 0.5.
    preciding_variables.remove('overall') # removes target variable.

    
    return preciding_variables  #return variables

high_corr = correlation(male_player_clean)
print('The preciding variables are:', high_corr)


The preciding variables are: ['movement_reactions', 'potential', 'passing', 'wage_eur', 'value_eur', 'dribbling']


## 3.Create and train a suitable machine learning model with cross-validation that can predict a player's rating. [5]

## Import Explanation
1. train_test_split and KFold from sklearn.model_selection:
   'trai_test_split': splits the dataset into training and testing sets.
   
   'KFold': provides cross-validaipn functionalit. it splits the dataset into 'K' folds for cross-validation, whcih helps in
    evaluationg mdoel mode robustly.
   
2. Metrics for Model Evaluation from sklearn.metrics:
   Mean Absolute Error: measures the average absolute errors between predicted and actual values.
   
   Mean Squared Error: measures the average squared errors between predicted and actual values.
   
   Mean Squared Log Error: measures the mean squared logarithmic errors between predicted and actual values.
   
   R2 scores: measures the proportion of variance in the depemndent that is predictable from the independent variables.
   
3. Regression Models:
   Linear Regression: this is a simple and widely uesed model for regression tasks

   Decision Tree Regressor: implements decision tree regression. Captures non-linear relationships in the data.

   XGBRegressor from 'xgboost': implements XGBoost regression. it implements gradient boosting .
   
6. Model Selection Tools:
   cros_val_score from 'sklearn.model,_selection': evaluares a model using cross validation by averaging the results of the multiple
   train-test splits.

   GridSearchCV from 'sklearn.model,_selection': performs grid search over specified parameter values for an estimator.

In [9]:
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV

In [10]:
X = male_player_clean[high_corr]
X = preprocessing(X)
# Select only the highly correlated features for the input features X
Y= male_player_clean['overall'] 

Xtrain,X_train_test,Ytrain,Y_train_test=train_test_split(X, Y,test_size=0.2,random_state=42)  # Splitting the data into training and testing sets

# Linear Regression Function BreakDown
This function trains and evaluates a linear regression model. it fits the mdoel to the training data and evaluates the predicitions on the test data using various metrics. 

Initialization and Fitting:

lin_reg = LinearRegression(): Initializes the Linear Regression model.

lin_reg.fit(X_train, Y_train): Fits the model to the training data.


Prediction:

y_pred = lin_reg.predict(X_test): Predicts the target values for the test data.
Evaluation Metrics:

mean_absolute_error(Y_test, y_pred): Computes the Mean Absolute Error between the actual and predicted values.

mean_squared_error(Y_test, y_pred): Computes the Mean Squared Error.

np.sqrt(mean_squared_error(Y_test, y_pred)): Computes the Root Mean Squared Error.

r2_score(Y_test, y_pred): Computes the R² score, which indicates the proportion of variance in the dependent variable that is predictable from the independent variables.

In [11]:
def linearReg(X, Y, Xt, Yt):
    lin_reg = LinearRegression()
    lin_reg.fit(X, Y)
    y_pred=lin_reg.predict(Xt)

    print(f"""
    Mean Absolute Error={mean_absolute_error(y_pred, Yt)}
    Mean Squared Error={mean_squared_error(y_pred,Yt)}
    Root Mean Squared Error={np.sqrt(mean_squared_error(y_pred, Yt))}
    R2 score={r2_score(y_pred, Yt)}
    """)
    return lin_reg


## Decision Tree Regression Function Breakdown
This function predicts continuous numerical values based on the structure of a decision tree. 

Initialization and Fitting:

dtree = DecisionTreeRegressor(): Initializes the Decision Tree Regressor model.

dtree.fit(X, Y): Fits the model to the training data.

mean_absolute_error(Yt, y_pred): Computes the Mean Absolute Error between the actual and predicted values.

mean_squared_error(Yt, y_pred): Computes the Mean Squared Error.

np.sqrt(mean_squared_error(Yt, y_pred)): Computes the Root Mean Squared Error.

r2_score(Yt, y_pred): Computes the R² score, which indicates the proportion of variance in the dependent variable that is predictable from the independent variables.

In [12]:
def desiciontree(X, Y, Xt, Yt):    
    dtree = DecisionTreeRegressor()
    dtree.fit(X,Y)
    y_pred_tree = dtree.predict(Xt) # predicts X_train_tes

    print(f"""
    Decision Tree regressor
    Mean Absolute Error={mean_absolute_error(y_pred_tree, Yt)}
    Mean Squared Error={mean_squared_error(y_pred_tree,Yt)}
    Root Mean Squared Error={np.sqrt(mean_squared_error(y_pred_tree, Yt))}
    R2 score={r2_score(y_pred_tree, Yt)}
    """) 
    return dtree

## Random Forest Regressor Function Breakdown

This function provides a consistent way to train and evaluate a Random Forest Regressor, making it easier to compare its performance with other models such as Linear Regression and Decision Tree Regressor.

Initialization and Fitting:

forest_model = RandomForestRegressor(): Initializes the Random Forest Regressor model.

forest_model.fit(X, Y): Fits the model to the training data.

Prediction:

mean_absolute_error(Y_test, y_pred): Computes the Mean Absolute Error between the actual and predicted values.

mean_squared_error(Y_test, y_pred): Computes the Mean Squared Error.

np.sqrt(mean_squared_error(Y_test, y_pred)): Computes the Root Mean Squared Error.

r2_score(Y_test, y_pred): Computes the R² score, which indicates the proportion of variance in the dependent variable that is predictable from the independent variables.

In [13]:
def randomforestreg(X, Y, Xt, Yt):
    forest_model=RandomForestRegressor()
    forest_model.fit(X,Y)
    y_pred_rand = forest_model.predict(Xt)
    
    print(f"""
    Random Forest Tree Regressor 
    Mean Absolute Error={mean_absolute_error(y_pred_rand, Yt)}
    Mean Squared Error={mean_squared_error(y_pred_rand,Yt)}
    Root Mean Squared Error={np.sqrt(mean_squared_error(y_pred_rand, Yt))}
    R2 score={r2_score(y_pred_rand, Yt)}
    """) 

    return forest_model

## XGBOOST Function Breakdown

Initialization and Fitting:

xg_model = XGBRegressor(): Initializes the XGB Regressor model.

xg_model.fit(X, Y): Fits the model to the training data.

Prediction:

mean_absolute_error(Y_test, y_pred): Computes the Mean Absolute Error between the actual and predicted values.

mean_squared_error(Y_test, y_pred): Computes the Mean Squared Error.

np.sqrt(mean_squared_error(Y_test, y_pred)): Computes the Root Mean Squared Error.

r2_score(Y_test, y_pred): Computes the R² score, which indicates the proportion of variance in the dependent variable that is predictable from the independent variables.

In [14]:
def Xgboostreg(X, Y,Xt,Yt):
    xgb_model = XGBRegressor()
    xgb_model.fit(X,Y)
    y_boost = xgb_model.predict(Xt)

    print(f"""
    XGBoost 
    Mean Absolute Error={mean_absolute_error(y_boost, Yt)}
    Mean Squared Error={mean_squared_error(y_boost,Yt)}
    Root Mean Squared Error={np.sqrt(mean_squared_error(y_boost, Yt))}
    R2 score={r2_score(y_boost, Yt)}
    """) 

    return xgb_model


## Cross Eval Function
it is designed to perform corss-valkidation on a given model, using a specidied datasaet and calculate the mean R2 score.

Cross-validation is a techique used to acquire the general performance of a model by dividing the data into mulitiple folds and training and testing the mdoel on these folds. 

@param:
    model - specified dataset
    
    X - Xtrain
    
    Y- Ytrain

@body
    cv - initializes the Kfold Cross Validator

    cv-scores - evaluates a model using cross-validation

'''prints out the r2 score of the mdoel'''

    

In [15]:
def crosseval(model, X, Y):
    # Initializes K-fold cross-validator
    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    #Perform cross-validation
    cv_scores = cross_val_score(model, X, Y, cv=cv, scoring='r2')

    #print mean R2 score 
    print(f'{model.__class__.__name__} CV mean R2 score: {cv_scores.mean()}')

    #return mean R2 score
    return cv_scores.mean()

## Hyper Tuning 
This function allows for flexible hyperparameter tuning of the Random Forest Regressor model, enhancing the overall performance and making it easier to test the model and its configuration.

@ param: model - specified dataset

body:

cv - initializes model parameter
param_grid-sccreates a dictionary of aparameters for tuning 

grid_search  - performs the serach over defined hyperparameter grid.

return grid_search.best_estimator_  - Returns the model with the best combination of hyperparameters found during the search

In [16]:
def hyperparameter_tuning(model):
    # Define hyperparameter grid based on the model type
    select = model

    if model.__class__.__name__ == 'RandomForestRegressor':
        # Hyperparameter tuning using GridSearchCV
        param_grid = {
            'n_estimators': [100],
            'max_depth': [10, 20],
            'min_samples_split': [5]
        }
    elif model.__class__.__name__ == 'XGBRegressor':
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 6, 9],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.6, 0.8, 1.0]
        }
    else:
        raise ValueError("Model not supported for hyperparameter tuning.")

        
    cv = KFold(n_splits=5, shuffle=True, random_state=42)  
    grid_search = GridSearchCV(select, param_grid, cv=cv, scoring='neg_mean_absolute_error')
    grid_search.fit(Xtrain, Ytrain)
    
    # return Best model
    return grid_search.best_estimator_

## Select Model Training Function
The 'select_model_training' function runs the various functions above sequentially to acquire an optimised model for fine tuning and testing with the test data set .

It consists of  if, elif statements to select a specific model for hyperparameter tuning.

In [17]:
def select_model_training(Xtrain, Ytrain, X_train_test,Y_train_test):
   
  
    linearReg(Xtrain, Ytrain, X_train_test,Y_train_test) #runs linear regression function 
    desiciontree(Xtrain, Ytrain, X_train_test, Y_train_test) # runs decision tree function
    randomforestreg(Xtrain, Ytrain, X_train_test, Y_train_test) #runs random forest regression function
    Xgboostreg(Xtrain, Ytrain, X_train_test, Y_train_test) # runs XGB regresssor function
    

    #cross evaluation
    lin_reg_crossval = crosseval(LinearRegression(),Xtrain, Ytrain) 
    random_forest_crossval = crosseval(RandomForestRegressor(),Xtrain, Ytrain)
    decision_crossval = crosseval(DecisionTreeRegressor(),Xtrain, Ytrain)
    Xgboost_crossval = crosseval(XGBRegressor(),Xtrain, Ytrain)

    '''The above code cross validates all the selected model  to acquire the R2 score of each model.'''
    
    if lin_reg_crossval > random_forest_crossval and lin_reg_crossval > decision_crossval and lin_reg_crossval > Xgboost_crossval:
        best_model = hyperparameter_tuning(LinearRegression())
    elif random_forest_crossval > lin_reg_crossval and random_forest_crossval > decision_crossval:
        best_model= hyperparameter_tuning(RandomForestRegressor())
    elif Xgboost_crossval > random_forest_crossval and Xgboost_crossval > decision_crossval and Xgboost_crossval >lin_reg_crossval:
        best_model = hyperparameter_tuning(XGBRegressor())
    else:
         best_model =hyperparameter_tuning(DecisionTreeRegressor())

    ''' The above code compares the R2 score of each model to find the one with theb highest, making it the best model for hyperparameter tuning '''
        
    
    return best_model

Xtest = update_22[high_corr]
Xtest = preprocessing(Xtest)
Ytest = update_22['overall']

In [18]:
optimised_model = select_model_training(Xtrain, Ytrain, X_train_test,Y_train_test) # runs the select_model_training function


    Mean Absolute Error=2.3098790047977418
    Mean Squared Error=8.882196688033224
    Root Mean Squared Error=2.980301442477459
    R2 score=0.7824134502104033
    

    Decision Tree regressor
    Mean Absolute Error=1.2651999463646584
    Mean Squared Error=4.724359315530526
    Root Mean Squared Error=2.1735591355034547
    R2 score=0.9056487999966436
    

    Random Forest Tree Regressor 
    Mean Absolute Error=1.0299038727377836
    Mean Squared Error=2.4139844625343416
    Root Mean Squared Error=1.5537002486111475
    R2 score=0.9490714354760422
    

    XGBoost 
    Mean Absolute Error=1.1560693259182686
    Mean Squared Error=2.571568776684252
    Root Mean Squared Error=1.6036111675478728
    R2 score=0.9454592893664154
    
LinearRegression CV mean R2 score: 0.820642989481388
RandomForestRegressor CV mean R2 score: 0.950491285382207
DecisionTreeRegressor CV mean R2 score: 0.9029280285104836
XGBRegressor CV mean R2 score: 0.9472851179653841


## Model Perfomance Function

The model_performance function contains the process of:

Training a machine learning model (model.fit(X, Y)),

Generating predictions for both the training and test sets (model.predict(X) and model.predict(Xt)),


Computing evaluation metrics (MAE and RMSE) to assess the model's performance on both sets,

Printing out these metrics in a clear and organized manner.


## 4.Measure the model's performance and fine-tune it as a process of optimization.

In [19]:
def model_performance(model, X, Y, Xt, Yt):
    # Fit the model
    model.fit(X, Y)
    
    # Evaluate the model
    Y_pred_train = model.predict(X) 
    Y_pred_test = model.predict(Xt)
    
    # Calculate metrics for training set
    train_mae = mean_absolute_error(Y, Y_pred_train) # MAE for training set
    train_rmse = np.sqrt(mean_squared_error(Yt,Y_pred_test))  # RMSE for training set
    
    # Calculate metrics for test set
    test_mae = mean_absolute_error(Yt, Y_pred_test) #MAE for training set
    test_rmse = np.sqrt(mean_squared_error(Yt,Y_pred_test))  # RMSE for test set

    
    print(f'Train MAE: {train_mae}')
    print(f'Train RMSE: {train_rmse}')
    print()
    print(f'Test MAE: {test_mae}')
    print(f'Test RMSE: {test_rmse}')

In [20]:
model_performance(optimised_model,Xtrain, Ytrain, X_train_test,Y_train_test) #calls model performance function

Train MAE: 0.5595931107503966
Train RMSE: 1.5446114054698585

Test MAE: 1.0320258925443428
Test RMSE: 1.5446114054698585


### 5. Use the data from another season(players_22) which was not used during the training to test how good is the model. [5]

## 5.Test Model Function
The test_model function is a straightforward  function used to evaluate a trained machine learning model on a separate test dataset (Xt, Yt)(i.e player_22 dataset).

In [21]:
def test_model(model, Xt, Yt):
    
    new_pred = model.predict(Xt)
    
    test_data_mae = mean_absolute_error(Yt, new_pred)
    test_data_rmse = np.sqrt(mean_squared_error(new_pred, Yt))
    
    print(f"Test data MAE: {test_data_mae}")
    print(f"Test data RMSE: {test_data_rmse}")

In [22]:
test_model(optimised_model, Xtest, Ytest)

Test data MAE: 0.763472377252185
Test data RMSE: 1.2135496497480753


In [23]:
import pickle as pkl # imports pickle library

In [24]:
# Save the trained model to a pickle file
with open('Fifa_Regression_Model.pkl', 'wb') as file:
    pkl.dump(optimised_model, file) #saves model as a pkl file

## 6. Deploy the model on a simple web page using either (Heroku, Streamlite, or Flask) and upload a video that shows how the model performs on the web page/site. [5]

1. To do this, I used the Streamlit library as it was the most straightforward and effective model for web app design using python.

2. The source code for the development will be provided in the GIThub repository
   
3. Here is the link to the deployed application : https://emmanueladoteyacquayefifapredict.streamlit.app/